In [69]:
import numpy as np
import re

from keras.layers import Dense, LSTM, Input, Embedding, Dropout
from keras.utils import np_utils
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import LambdaCallback

In [21]:
token_type = 'word'

In [57]:
#load in the text and perform some cleanup

filename = "./data/text/aesop.txt"

with open(filename, encoding='utf-8-sig') as f:
    text = f.read()
    
text = text.lower()
text = text.replace('\n\n\n\n\n', '|')
text = text.replace('\n', ' ')
text = re.sub('  +', '. ', text).strip()
text = text.replace('..', '.')

text = re.sub('([!"#$%&()*+,-./:;<=>?@[\]^_`{|}~])', r' \1 ', text)
text = re.sub('\s{2,}', ' ', text)



In [58]:
len(text)

202881

In [59]:
text

'the fox and the grapes . a hungry fox saw some fine bunches of grapes hanging from a vine that was trained along a high trellis , and did his best to reach them by jumping as high as he could into the air . but it was all in vain , for they were just out of reach : so he gave up trying , and walked away with an air of dignity and unconcern , remarking , " i thought those grapes were ripe , but i see now they are quite sour . " | the goose that laid the golden eggs . a man and his wife had the good fortune to possess a goose which laid a golden egg every day . lucky though they were , they soon began to think they were not getting rich fast enough , and , imagining the bird must be made of gold inside , they decided to kill it in order to secure the whole store of precious metal at once . but when they cut it open they found it was just like any other goose . thus , they neither got rich all at once , as they had hoped , nor enjoyed any longer the daily addition to their wealth . much 

In [1]:

if token_type == 'word':
    tokenizer = Tokenizer(char_level = False, filters = '')
    seq_length = 20
else:
    tokenizer = Tokenizer(char_level = True, filters = '', lower = False)
    seq_length = 50
    
    
tokenizer.fit_on_texts([text])

total_words = len(tokenizer.word_index) + 1

token_list = tokenizer.texts_to_sequences([text])[0]


NameError: name 'token_type' is not defined

In [63]:
tokenizer.word_index

{',': 1,
 'the': 2,
 'and': 3,
 '.': 4,
 'a': 5,
 'to': 6,
 '"': 7,
 'of': 8,
 'he': 9,
 'his': 10,
 'was': 11,
 'you': 12,
 'in': 13,
 'him': 14,
 'it': 15,
 'for': 16,
 'i': 17,
 'but': 18,
 'that': 19,
 'with': 20,
 'said': 21,
 '|': 22,
 'they': 23,
 'at': 24,
 'as': 25,
 'when': 26,
 'on': 27,
 'so': 28,
 'by': 29,
 ':': 30,
 'one': 31,
 'had': 32,
 ';': 33,
 'be': 34,
 'them': 35,
 'up': 36,
 'all': 37,
 'who': 38,
 '-': 39,
 'her': 40,
 'my': 41,
 'is': 42,
 'out': 43,
 'me': 44,
 'not': 45,
 'your': 46,
 'were': 47,
 'which': 48,
 'have': 49,
 'their': 50,
 'no': 51,
 'lion': 52,
 'she': 53,
 'if': 54,
 'into': 55,
 'fox': 56,
 'an': 57,
 'what': 58,
 'ass': 59,
 '!': 60,
 'time': 61,
 'from': 62,
 'are': 63,
 'came': 64,
 '?': 65,
 'man': 66,
 'do': 67,
 'about': 68,
 'there': 69,
 'wolf': 70,
 'then': 71,
 'upon': 72,
 'himself': 73,
 'this': 74,
 'day': 75,
 'would': 76,
 'some': 77,
 'once': 78,
 'will': 79,
 'very': 80,
 'went': 81,
 'well': 82,
 'than': 83,
 'down': 84,
 

In [65]:
step = 1

def generate_sequences(token_list):
    
    X = []
    y = []

    
    for i in range(0, len(token_list) - seq_length, step):
        X.append(token_list[i: i + seq_length])
        y.append(token_list[i + seq_length])
    

    y = np_utils.to_categorical(y, num_classes = total_words)
    
    num_seq = len(X)
    print('Number of sequences:', num_seq, "\n")
    
    return X, y, num_seq


X, y, num_seq = generate_sequences(token_list)

X = np.reshape(X, (num_seq, seq_length))


Number of sequences: 44999 



In [66]:
total_words

4169

In [70]:
# define the LSTM model

n_units = 256
embedding_size = 256

text_in = Input(shape = (seq_length,))
x = Embedding(total_words, embedding_size, input_length=seq_length)(text_in)
# x = LSTM(n_units, return_sequences = True )(x)
# x = Dropout(0.2)(x)
x = LSTM(n_units)(x)
x = Dropout(0.2)(x)
text_out = Dense(total_words, activation = 'softmax')(x)

model = Model(text_in, text_out)

opti = RMSprop(lr = 0.001)
# opti = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opti)

In [76]:
def sample_with_temp(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



def generate_text(seed_text, next_words, model, max_sequence_len, temp):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='pre')
        token_list = np.reshape(token_list, (1, max_sequence_len))

        probs = model.predict(token_list, verbose=0)[0]
        y_class = sample_with_temp(probs, temperature = temp)
#         y_class = np.random.choice(total_words,  p = probs)
        
        if y_class == 0:
            output_word = ''
        else:
            output_word = tokenizer.index_word[y_class]
            
        if token_type == 'word':
            seed_text += ' ' + output_word
        else:
            seed_text += output_word
            
        if output_word == "|":
            break
            
    return seed_text

In [77]:
seed_text = "|"
gen_words = 100

def on_epoch_end(epoch, logs):
    print('Temp 0.2')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.2))
    print('Temp 0.5')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.5))
    print('Temp 1.0')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 1))

In [78]:
callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [79]:
epochs = 1000
batch_size = 32
num_batches = int(len(X) / batch_size)
model.fit(X, y, epochs=epochs, batch_size=batch_size, callbacks = [callback], shuffle = True)

Epoch 1/1000
44999/44999 [==============================] - 86s 2ms/step - loss: 4.6704
Temp 0.2
| the lion and the lion . a man who was once . a man who was a man who was a man who had a man who was , and the lion , and the lion , and the boy was a man who was to the lion , and the lion , and the lion , and the boy was a man who was , and the boy was a man who had a mouse , and the man who had a man who had a man who had a man who was , and the boy was a
Temp 0.5
| the edge of the lamb , and his master , and the boy was gathering a man who had a man who found to make |
Temp 1.0
| the one the ass and the frog run as that , and all a purpose was a lion - working . |
Epoch 2/1000
44999/44999 [==============================] - 88s 2ms/step - loss: 4.4692
Temp 0.2
| the wolf and the wolf . a farmer was the wolf , and the fox , and the fox , and the fox , and the fox , and the fox , and the fox , and the ass went to be a beasts . a hungry fox , and the fox , and the wolf , and the fox , and

/Users/davidfoster/.virtualenvs/gdl/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


| the lion and the wolf . a stag , and his axe . a farmer was so pleased , and the lion , and said , " i do you see what you may be done , " said the frog , " i'll wake them . the dog , and the ass , and the wolf . " the wolf , " said the fox , " said he said , " i am useful for some heat of the habit of a world , and the lion , who pretend to die with his feet . in the
Temp 1.0
| the done . a hawk seemed disarmed ? by your fine ready for the fox . the farmer , and went hunting , and acrobats long come on a quantity of the other where he said , " it's not it up thing at the king of him and said , " i'm not friends , " snapped the evil did not sure he ? think he was found a rest in good will put . " the leaves of two will no , said , " do you call you , he cried , " no want well ?
Epoch 6/1000
44999/44999 [==============================] - 91s 2ms/step - loss: 4.1326
Temp 0.2
| the lion and the lion . a man once bought a lion and the lion crow , and a ass and the fox , and the lion , an

| the ass and was just then a large a horse , and the fox , and a lion was a man fell the woodman in the lion much less by the dolphin . |
Temp 1.0
| the thief . a bull accepted in a poor he , when the lion felt hardly got up the wolf were walking along a dead and the lion's footprints neighbourhood of don't when it seemed as before at this he , and a wants and every time of her a badge of wind . |
Epoch 15/1000
44999/44999 [==============================] - 94s 2ms/step - loss: 3.8584
Temp 0.2
| the ass and the fox . a lion was a lion , and the lion , and the ass , and the other the anger of the land and said , " i have been thrown a good deal - house contained a lion was so hard at the lion , and the ass , and the ass , and the lion . " |
Temp 0.5
| the ass , and the forest . the wolf , the wild goats approach of the sound of mankind , " and said , " i have been , " said he , " said he , " said he , " said he , " i have been like a wolf , and the lion was a very good deal more tests 

44999/44999 [==============================] - 88s 2ms/step - loss: 3.7613
Temp 0.2
| the bald man and the fox and the fox and the fox and the fox and the cat . " |
Temp 0.5
| the bald man . " the wolf and the fox , and , and a tree ran out of his own life to the frog made a summer's himself above the other dangers at the branches of his breast , while the butcher's condition of its life . " |
Temp 1.0
| the blacksmith , until at last him home on the wolf . however , " they found she care whether the ass's shadow . " the hawk . " they'd pat me first he was to the other , " what was brought in an appearance that i am going to me : you stay of the of himself : he's no match for her nest at them ; so , and his friend and a man . " one of juno i am going to eat likely to be relit as much ; but we have lost my as
Epoch 24/1000
44999/44999 [==============================] - 89s 2ms/step - loss: 3.7457
Temp 0.2
| the ass and the fox and the fox and the ass and the ass and the ass and the lion

...

44999/44999 [==============================] - 98s 2ms/step - loss: 3.5777
Temp 0.2
| the lion and the lion . a man once a dog , and the fox , and the ass was the agonies of my subjects ? " i am not recognise him . " the claimant , and said to him , " said the fox , and the fox , and the meat and the meat and the thief , and the fox , and the same time whining and lifting up his companions , and , and the rooks and the rooks and the rooks and the rooks and , and the fox only means , and said
Temp 0.5
| the lion and the lion . a man once a dog , and the rooks and the fox . a goat was overjoyed at recovering his property , and dusty back , and the harness , and the same you were , and the fox , and the most honest and the farm when i have been up find what . " you feed were which it was not present like the dog , and the lion . |
Temp 1.0
| the rooks and the lion . a man who stole to die against such didn't fled at our outnumbered by their was stretching himself up from every leaf . the 

44999/44999 [==============================] - 99s 2ms/step - loss: 3.6067
Temp 0.2
| the lion and the wolf . a fox , and the fox settled in the forest the tree , and the same farmyard ; and the other foxes at the fox and the monkey replied that he was foolish enough to coax them to him , " said he meant nothing to the of the day , " said the fox , and the other foxes of his friends . " |
Temp 0.5
| the lion and the lion and the fox his burdens , and the lion . the lion divided it , and the day and said , " i am going to give the , " said he ? " i am what cried , " for i have never yet i am not only can't do so i am grateful with a voice like mine their own . " |
Temp 1.0
| the wolf , so nimble by the hawk was overjoyed at the ants stopped him for him and whenever he said , " |
Epoch 59/1000
44999/44999 [==============================] - 99s 2ms/step - loss: 3.5333
Temp 0.2
| the ass and the snake . a wolf was war with the sea , and the stag . " |
Temp 0.5
| the fowler , and the rest .

| the boy bathing . a man once bought , even one your poor feathers ! you are you no ! i am saved my vow ? " i thought those grapes your necks , ordered it seems he's not clever enough to recognise and hearing never noticed that it fell into a hard in a farmyard , " at once detected a sweet as i content a place where my friend supperless to entrust him off . " " the ass of his good , and cawed for him , the thief . what do you mean it what it
Epoch 69/1000
44999/44999 [==============================] - 99s 2ms/step - loss: 3.4854
Temp 0.2
| the time , and the fox replied , " i am saved from the power of a summer's day . " |
Temp 0.5
| the eagle and the other replied , " i know very nice dry weather , and was just as he was about the greatest . the appointed day in the ass , and the other animal was afraid of the mouse and the ass picked him with him , and said to himself , and the other animal that the tortoise he had with him with him , and the time it was to the ass only looked a gen

 4800/44999 [==>...........................] - ETA: 1:20 - loss: 3.1578

KeyboardInterrupt: 

In [2]:
print (generate_text(seed_text, 200, model, seq_length))

NameError: name 'generate_text' is not defined